In [225]:
import pandas as pd
import numpy as np

In [226]:
def resample_and_interpolate(group):
    return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()

In [227]:
weather_df = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/weather_data/DWD_ICON-EU.csv")
solar_total = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/basic_files/solar_total_production.csv")
weather_df.sort_values(by='ref_datetime', inplace=True)
weather_df = weather_df.groupby(["valid_datetime","latitude","longitude"]).last().reset_index()
weather_df.reset_index(inplace=True)

In [228]:
weather_df.valid_datetime = pd.to_datetime(weather_df.valid_datetime)
weather_df = weather_df.set_index(["valid_datetime","latitude","longitude"])
df_resampled = weather_df.groupby(['latitude', 'longitude'], group_keys=False).apply(resample_and_interpolate)
df_resampled = df_resampled.reset_index()

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_8446/2834871732.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_8446/2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_8446/2834871732.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_8446/2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecat

In [229]:
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
df_resampled.drop(columns=['index','ref_datetime'], inplace=True)
df_resampled_merged = pd.merge(df_resampled, solar_total, how='left', left_on='valid_datetime', right_on='timestamp_utc')
df_resampled_merged_solar = df_resampled_merged.loc[~(df_resampled_merged.latitude == 53.935) & ~(df_resampled_merged.longitude == 1.8645)]

In [230]:
df_resampled_merged_solar1 = df_resampled_merged_solar.groupby("valid_datetime").mean().reset_index()

In [231]:
distinct_lat_lon_pairs = df_resampled_merged_solar[['latitude', 'longitude']].drop_duplicates()

In [232]:
df_resampled_merged_solar1.head()

,valid_datetime,latitude,longitude,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,PressureReducedMSL,SolarDownwardRadiation,TotalPrecipitation,timestamp_utc,generation_mw,installed_capacity_mwp,capacity_mwp
0,2024-10-07 00:00:00+00:00,52.381681,0.650333,14.12700,4.81800,8.3690,226.31250,0.89300,93.61300,99466.75350,0.0,NaN,2024-10-07 00:00:00+00:00,0.0,2956.745251,2779.542388
1,2024-10-07 00:30:00+00:00,52.381681,0.650333,13.96225,4.93950,8.5185,232.54025,0.89825,92.78875,99514.00425,0.0,NaN,2024-10-07 00:30:00+00:00,0.0,2956.745251,2779.542384
2,2024-10-07 01:00:00+00:00,52.381681,0.650333,13.79750,5.06100,8.6680,238.76800,0.90350,91.96450,99561.25500,0.0,0.0135,2024-10-07 01:00:00+00:00,0.0,2956.745251,2779.542383
3,2024-10-07 01:30:00+00:00,52.381681,0.650333,13.53700,5.07225,8.7065,239.91850,0.86175,91.31550,99557.52450,0.0,0.0090,2024-10-07 01:30:00+00:00,0.0,2956.745251,2779.542370
4,2024-10-07 02:00:00+00:00,52.381681,0.650333,13.27650,5.08350,8.7450,241.06900,0.82000,90.66650,99553.79400,0.0,0.0045,2024-10-07 02:00:00+00:00,0.0,2956.745251,2779.542348


In [233]:
def set_up_solar_features(df):
    df["cos_day"] = np.cos(2 * np.pi * df.valid_datetime.dt.dayofyear / 365)
    df["cos_hour"] = np.cos(2 * np.pi * df.valid_datetime.dt.hour / 24)
    df["SolarDownwardRadiation_Mean"] = df.SolarDownwardRadiation
    df["Temperature_dwd_Mean"] = df.Temperature
    df["Temperature_dwd_Std"] = df_resampled_merged_solar.groupby("valid_datetime").std().reset_index().Temperature
    df["SolarDownwardRadiation_RW_dwd_Mean_30min"] = df.SolarDownwardRadiation_Mean.rolling(window=1, min_periods=1).mean()
    df["SolarDownwardRadiation_RW_Mean_1h"] = df.SolarDownwardRadiation_Mean.rolling(window=2, min_periods=1).mean()
    df["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df.SolarDownwardRadiation_Mean.shift(1)
    df["SolarDownwardRadiation_Mean_Lag_1h"] = df.SolarDownwardRadiation_Mean.shift(2)
    df["SolarDownwardRadiation_Mean_Lag_24h"] = df.SolarDownwardRadiation_Mean.shift(48)
    for i in range(len(distinct_lat_lon_pairs)):
        lat = distinct_lat_lon_pairs.latitude.iloc[i]
        lon = distinct_lat_lon_pairs.longitude.iloc[i]
        mask = (df_resampled_merged_solar.latitude == lat) & (df_resampled_merged_solar.longitude == lon)
        df[f"Temperature_{i}"] = pd.Series(df_resampled_merged_solar.Temperature[mask].values)[:len(df)]  # Fill gaps with NaN
        df[f"SolarDownwardRadiation_{i}"] = pd.Series(df_resampled_merged_solar.SolarDownwardRadiation[mask].values)[:len(df)]  # Fill gaps with NaN
    return df
df_resampled_merged_solar2 = set_up_solar_features(df_resampled_merged_solar1)

In [234]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [235]:
for i in range(20):
    temp_col = f'Temperature_{i}'
    irradiance_col = f'SolarDownwardRadiation_{i}'
    panel_temp_col = f'Panel_Temperature_Point{i}'
    panel_eff_col = f'Panel_Efficiency_Point{i}'
    df_resampled_merged_solar2[panel_temp_col], df_resampled_merged_solar2[panel_eff_col] = pv_temperature_efficiency(df_resampled_merged_solar2[irradiance_col], df_resampled_merged_solar2[temp_col])

In [236]:
df_resampled_merged_solar2["Panel_Temperature_Mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_Mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Temperature_Std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").std(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_Std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").std(axis= 1)

In [237]:
df_resampled_merged_solar2["Solar_MWh_Lag_48h"] = df_resampled_merged_solar2.generation_mw.shift(periods= 96) * 0.5
df_resampled_merged_solar2["Solar_MWh_credit"] = df_resampled_merged_solar2.generation_mw * 0.5
df_resampled_merged_solar2["Capacity_MWP_Lag_48h"] = df_resampled_merged_solar2.capacity_mwp.shift(periods= 96)

In [238]:
df_resampled_merged_solar3 = df_resampled_merged_solar2[[ 
    "SolarDownwardRadiation_Mean",
    "SolarDownwardRadiation_RW_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_Mean_Lag_1h",
    "SolarDownwardRadiation_Mean_Lag_24h",
    "Panel_Efficiency_Mean",
    "Panel_Efficiency_Std",
    "Panel_Temperature_Mean",
    "Panel_Temperature_Std",
    "Temperature_dwd_Std",
    "Temperature_dwd_Mean",
    "cos_hour",
    "cos_day",
    "Capacity_MWP_Lag_48h",
    "Solar_MWh_Lag_48h",
    "Solar_MWh_credit"
    ]]
df_resampled_merged_solar3.dropna(inplace=True)

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_8446/2103203449.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resampled_merged_solar3.dropna(inplace=True)


In [239]:
df_resampled_merged_solar3.columns

Index(['SolarDownwardRadiation_Mean', 'SolarDownwardRadiation_RW_Mean_1h',
       'SolarDownwardRadiation_RW_dwd_Mean_30min',
       'SolarDownwardRadiation_dwd_Mean_Lag_30min',
       'SolarDownwardRadiation_Mean_Lag_1h',
       'SolarDownwardRadiation_Mean_Lag_24h', 'Panel_Efficiency_Mean',
       'Panel_Efficiency_Std', 'Panel_Temperature_Mean',
       'Panel_Temperature_Std', 'Temperature_dwd_Std', 'Temperature_dwd_Mean',
       'cos_hour', 'cos_day', 'Capacity_MWP_Lag_48h', 'Solar_MWh_Lag_48h',
       'Solar_MWh_credit'],
      dtype='object')

In [240]:
df_resampled_merged_solar3.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/test.csv", index=False)

## Test normalize target

In [241]:
import numpy as np

df_resampled_merged_solar2["Target_Capacity_MWP_%"] = np.where(df_resampled_merged_solar2["Solar_MWh_credit"] == 0.0, 0, df_resampled_merged_solar2["Solar_MWh_credit"] / df_resampled_merged_solar2["capacity_mwp"])
df_resampled_merged_solar2["Target_Capacity_MWP_%_Lag_48"] = df_resampled_merged_solar2["Target_Capacity_MWP_%"].shift(96)

In [245]:
df_resampled_merged_solar2["capacity_mwp"].mean()

np.float64(2779.336573478634)

In [242]:
df_resampled_merged_solar4 = df_resampled_merged_solar2[[ 
    "SolarDownwardRadiation_Mean",
    "SolarDownwardRadiation_RW_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_Mean_Lag_1h",
    "SolarDownwardRadiation_Mean_Lag_24h",
    "Panel_Efficiency_Mean",
    "Panel_Efficiency_Std",
    "Panel_Temperature_Mean",
    "Panel_Temperature_Std",
    "Temperature_dwd_Std",
    "Temperature_dwd_Mean",
    "cos_hour",
    "cos_day",
    "Solar_MWh_Lag_48h",
    "Capacity_MWP_Lag_48h",
    "Target_Capacity_MWP_%_Lag_48",
    "Target_Capacity_MWP_%",
    ]]
df_resampled_merged_solar4.dropna(inplace=True)

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_8446/1790439630.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resampled_merged_solar4.dropna(inplace=True)


In [243]:
df_resampled_merged_solar4

,SolarDownwardRadiation_Mean,SolarDownwardRadiation_RW_Mean_1h,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_Mean_Lag_1h,SolarDownwardRadiation_Mean_Lag_24h,Panel_Efficiency_Mean,Panel_Efficiency_Std,Panel_Temperature_Mean,Panel_Temperature_Std,Temperature_dwd_Std,Temperature_dwd_Mean,cos_hour,cos_day,Solar_MWh_Lag_48h,Capacity_MWP_Lag_48h,Target_Capacity_MWP_%_Lag_48,Target_Capacity_MWP_%
96,0.0,0.0,0.0,0.0,0.0,0.0,0.188081,0.000357,13.776000,0.496404,0.337606,13.58750,1.000000,0.158559,0.0,2779.542388,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.188098,0.000327,13.753000,0.454666,0.349814,13.48025,1.000000,0.158559,0.0,2779.542384,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.188098,0.000327,13.753000,0.454666,0.370293,13.37300,0.965926,0.158559,0.0,2779.542383,0.0,0.0
99,0.0,0.0,0.0,0.0,0.0,0.0,0.188134,0.000287,13.702750,0.399291,0.364783,13.32325,0.965926,0.158559,0.0,2779.542370,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.188134,0.000287,13.702750,0.399291,0.362641,13.27350,0.866025,0.158559,0.0,2779.542348,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,0.0,0.0,0.0,0.0,0.0,0.0,0.186083,0.000531,16.551234,0.737510,0.521161,6.64250,0.707107,0.242850,0.0,2779.218524,0.0,0.0
343,0.0,0.0,0.0,0.0,0.0,0.0,0.185100,0.000825,17.916750,1.145369,0.556076,6.72100,0.707107,0.242850,0.0,2779.218512,0.0,0.0
344,0.0,0.0,0.0,0.0,0.0,0.0,0.185474,0.000665,17.397836,0.923327,0.602857,6.79950,0.500000,0.242850,0.0,2779.218480,0.0,0.0
345,0.0,0.0,0.0,0.0,0.0,0.0,0.185847,0.000667,16.878922,0.926172,0.678056,6.93050,0.500000,0.242850,0.0,2779.218479,0.0,0.0


In [244]:
df_resampled_merged_solar4.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/test_norm.csv", index=False)